# Final results

In [ ]:
import pandas as pd
import pickle
import joblib

In [ ]:
test_df = pd.read_csv('../data/BipolarDerivedDataTesting.csv')
test_df.head()

### Import Kerry's model for bp prediction

In [ ]:
with open('../models/kerry-pipeline1_prediction.pkl', 'rb') as f:
    search = pickle.load(f)
kerry_test_df = test_df.filter(regex=("t1r_.*m"))
pred_arr = search.predict(kerry_test_df)
pred_arr

In [ ]:
kerry_df = pd.DataFrame()
case_control = [1 if pred == 'Case' else 0 for pred in list(pred_arr)]
case_control.count(1)/len(case_control)

In [ ]:
kerry_df['ID'] = test_df['record_id']
kerry_df['case_control'] = case_control
kerry_df.to_csv('../data/kerry-bp_hackathon_submission.txt', index=False)

### Import James' predictions

In [ ]:
with open('../models/jdkent-pipeline1_proba.pkl', 'rb') as f:
    james_bp_pred1 = joblib.load(f)
f.close()

with open('../models/jdkent-pipeline2_proba.pkl', 'rb') as f:
    james_bp_pred2 = joblib.load(f)
f.close()

with open('../models/jdkent-ymrs_prediction.pkl', 'rb') as f:
    james_ymrs_pred = joblib.load(f)
f.close()

with open('../models/jdkent-madrs_prediction.pkl', 'rb') as f:
    james_madrs_pred = joblib.load(f)
f.close()

In [ ]:
james_df = pd.DataFrame()
james_df['ID'] = test_df['record_id']
james_avg_bp_probas = list(zip(((james_bp_pred1[:, 0] + james_bp_pred2[:, 0])/2), ((james_bp_pred1[:, 1] + james_bp_pred2[:, 1])/2)))
james_bp_pred = [1 if ele[0]>ele[1] else 0 for ele in james_avg_bp_probas]
james_df['case_control'] = james_bp_pred
james_df['YMRS'] = james_ymrs_pred
james_df['MADRS'] = james_madrs_pred

In [ ]:
james_df.to_csv('../data/jdkent-bp_hackathon_submission.txt', index=False)

### Import Zoheb's predictions

In [ ]:
zoheb_df = pd.read_csv('../data/zkhan1-bp_hackathon_submission.txt')

## Get final bp predictions

In [4]:
james_df = pd.read_csv('../data/jdkent-bp_hackathon_submission.txt')
zoheb_df = pd.read_csv('../data/zkhan1-bp_hackathon_submission.txt')
kerry_df = pd.read_csv('../data/kerry-bp_hackathon_submission.txt')

In [5]:
final_df = pd.DataFrame()
final_df['ID'] = zoheb_df['ID']
final_df['case_control'] = list((0.4*james_df['case_control'] + 0.3*zoheb_df['case_control'] + 0.3*kerry_df['case_control'] >.49) *1)
final_df['YMRS'] = list((0.5*james_df['YMRS'] + 0.5*zoheb_df['YMRS']))
final_df['MADRS'] = list((0.5*james_df['MADRS'] + 0.5*zoheb_df['MADRS']))

In [6]:
final_df.head()
list(final_df[final_df.loc[:, 'ID'] == 1028].loc[:, ['case_control', 'YMRS', 'MADRS']].values.squeeze())

[0.0, 0.5470308505021286, 2.893171071263428]

In [7]:
import numpy as np
import pandas as pd

template = pd.read_csv('../data/submission_template.txt')

In [8]:
template.columns = ['ID']
template['case_control'] = np.nan
template['YMRS'] = np.nan
template['MADRS'] = np.nan
for index, row in template.iterrows():
    template.loc[index, ['case_control', 'YMRS', 'MADRS']] = final_df[final_df.loc[:, 'ID'] == row['ID']].loc[:, ['case_control', 'YMRS', 'MADRS']].values.squeeze()

In [12]:
template['case_control'] = template['case_control'].astype(int)
template

,ID,case_control,YMRS,MADRS
0,1028,0,0.547031,2.893171
1,1063,0,0.406909,3.412418
2,1089,0,0.571636,6.414442
3,1100,1,0.742284,6.016543
4,1114,0,0.723972,7.207083
5,1186,1,0.137479,4.848300
6,1188,1,0.487705,6.836747
7,1192,0,0.658280,6.744404
8,1197,1,0.189431,3.323930
9,1222,0,0.337850,4.492498


In [13]:
template.to_csv('../data/final-bp_hackathon_submission.txt', index=False)